# 0. 라이브러리 임포트

In [14]:
#%pip install selenium
%pip install gensim
%pip install pyLDAvis

# 크롤링
import bs4
from bs4 import BeautifulSoup
import pandas as pd
import numpy
import warnings
import re
import requests
import urllib
import time
import selenium
from selenium.webdriver.common.by import By
from selenium import webdriver
import urllib
from urllib.request import urlopen 

#options = webdriver.ChromeOptions()
#options.add_argument('headless') # 옵션 적용 시 에러 발생
driver = webdriver.Chrome()
warnings.filterwarnings(action='ignore')

# 시각화 라이브러리
import matplotlib.pyplot as plt
from matplotlib import font_manager, rc
from wordcloud import WordCloud
from collections import Counter
import  numpy as np
from PIL import Image

# 폰트 설정
font_path = r'C:\\windows\\fonts\\nanumgothicbold.ttf'
font = font_manager.FontProperties(fname = font_path).get_name()
rc('font',family = font)


# 자연어처리
from konlpy import *
from konlpy.tag import *
kom = Komoran()


# LDA 라이브러리
import gensim
from gensim import corpora
import pyLDAvis.gensim_models 



Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


# 1. 크롤링
- 개요, 연혁, 교육이념, 교육목표, 전공소개, 학과 안내, 강의교과목
- 개별 페이지를 문서 하나로 봄

## 1-1. 중앙대학교

In [17]:
# 세부 페이지 접속 함수
def go_to(dept1_idx, dept2_idx):
    driver = webdriver.Chrome()
    driver.get("https://lis.cau.ac.kr/")
    time.sleep(1)
    driver.find_element('xpath','//*[@id="gnavigation"]').click()
    driver.find_element('xpath','//*[@id="gnavigation"]/ul/li[1]/a').click()
    driver.find_element('xpath','//*[@id="gnavigation"]/ul/li[1]/div/div/ul/li[%s]/a[%s]'%(dept1_idx, dept2_idx)).click()
    return driver

In [18]:
# 크롤링
cau = ' '

# 개요 / 연혁
driver = go_to(1,1)
c_1 = ' '
cont1 = driver.find_element('xpath','//*[@id="contents"]/dl[1]/dd[4]').text
cont2 = driver.find_element('xpath','//*[@id="contents"]/dl[1]/dd[6]').text
cau = cau + cont1+cont2
c_1 = cont1 + cont2 
driver.close()

# 교육 이념
driver = go_to(1,2)
c_2 = ' '
cont3 = driver.find_element('xpath', '//*[@id="contents"]/dl[1]/dd[3]/span').text
cont4 = driver.find_element('xpath','//*[@id="contents"]/dl[1]/dd[6]/span').text
cont5 = driver.find_element('xpath','//*[@id="contents"]/dl[1]/dd[8]/span').text
cont6 = driver.find_element('xpath','//*[@id="contents"]/dl[1]/dd[10]/span').text
cont7 = driver.find_element('xpath','//*[@id="contents"]/dl[1]/dd[12]').text
cont8 = driver.find_element('xpath','//*[@id="contents"]/dl[1]/dd[14]/span').text
cau = cont3+cont4+cont5+cont6+cont7+cont8+cau
c_2 = cont3+cont4+cont5+cont6+cont7+cont8
driver.close()

# 교과 과정
temp_li=[]
driver = go_to(3,3)
c_3 = ' '
td = driver.find_elements('xpath','//*[@id="contents"]/dl[1]/table[1]/tbody')
td = td[0].text.split('\n')
temp_li += td
for x in range(1,5,1):
    text= driver.find_element('xpath','//*[@id="contents"]/dl[1]/div[%s]'%str(x)).text
    temp_li.append(text)
for x in temp_li:
    cau += x
    c_3 += x
driver.close()

## 1-2. 연세대학교

In [19]:
# 옵션
options = webdriver.ChromeOptions()
options.add_argument('headless') # 옵션 적용 시 에러 발생

# 세부 페이지 접속 함수
def go_to_y(dept1_idx, dept2_idx):
    driver = webdriver.Chrome(options = options)
    driver.get("https://yslis.yonsei.ac.kr/doai/index.do")
    time.sleep(1)
    
    # 마우스 오버 
    action_dir = driver.find_element('xpath','//*[@id="item_body"]/header/div[2]/div/div/ul/li[%s]/a'%dept1_idx)
    act = webdriver.ActionChains(driver)
    act.move_to_element(action_dir).perform() # 마우스 오버 시켜야 상세 메뉴 나타남
    act.perform()

    # 요소 찾기
    driver.find_element('xpath','//*[@id="item_body"]/header/div[2]/div/div/ul/li[%s]/ul/li[%s]/a'%(dept1_idx, dept2_idx)).click()
    return driver

In [20]:
# 크롤링
yon = ''

# 학과 소개
y_1 = ' '
driver  = go_to_y(1,1)
t = driver.find_element('xpath','//*[@id="jwxe_main_content"]/div').text
yon += t
y_1 += t
driver.close()

# 학과 연혁
y_2 = ' '
driver = go_to_y(1,2)
t = driver.find_element('xpath','//*[@id="his2000"]').text
yon += t
driver.find_element('xpath','//*[@id="jwxe_main_content"]/div/div/div[1]/ul/li[2]/a').click()
t2 = driver.find_element('xpath','//*[@id="his1990"]').text
yon += t2
y_2 += t2
y_2 += t
driver.close()

# 교과 과정
y_3 = ' '
driver = go_to_y(3,1)
t = driver.find_element('xpath','//*[@id="jwxe_main_content"]').text
re_comp  = re.compile('[A-Z]{3}[0-9]{4}') # 과목코드 제거
skk = re_comp.sub('',yon)
yon += t
y_3 += t
driver.close()


## 1-3. 성균관대학교

In [21]:
# 세부 페이지 접속 함수
def go_to_s(dept1_idx, dept2_idx):
    driver = webdriver.Chrome()  # 창 제거 옵션 선택 시 에러 발생
    driver.get("https://lis.skku.edu/lis/index.do")
    driver.maximize_window() # 창을 최대화 했을 때 메뉴의 태그가 달라짐
    time.sleep(1)
    
    # 마우스 오버 
    action_dir = driver.find_element('xpath','//*[@id="item_body"]/header/div[2]/div[2]/div/div[2]/div/ul/li[%s]/a'%dept1_idx)
    act = webdriver.ActionChains(driver)
    act.move_to_element(action_dir).perform() # 마우스 오버 시켜야 상세 메뉴 나타남
    act.perform()

    # 요소 찾기
    driver.find_element('xpath','//*[@id="item_body"]/header/div[2]/div[2]/div/div[2]/div/ul/li[%s]/ul/li[%s]/a'%(dept1_idx, dept2_idx)).click()
    return driver


In [22]:
# 크롤링
skk = ' '

# 인사말  - 준비중

# 학과 연혁
s_1 = ' '
driver = go_to_s(1,2)
t = driver.find_element('xpath', '//*[@id="jwxe_main_content"]').text
skk += t
s_1 += t
driver.close()

# 교육 목표 및 학과 안내
s_2 = ' '
driver = go_to_s(1,3)
t = driver.find_element('xpath','//*[@id="jwxe_main_content"]').text
skk += t
s_2 += t
driver.close()

# 교육과정
driver = go_to_s(2,1)
s_3 = ' '
for i in range(2,5,1):
    t = driver.find_element('xpath','//*[@id="jwxe_main_content"]/div[1]/div/table/tbody').text
    skk += t
    s_3 += t
    driver.find_element('xpath','//*[@id="jwxe_main_content"]/div[2]/ul/li[%s]'%i).click()

# 과목코드, 개설여부 -> 전처리 단계에서 다른 학교들과 한번에 처리하려면 복잡해짐 -> 미리 전처리
skk = skk.replace('Yes','')
skk = skk.replace('No','')
s_3 = s_3.replace('Yes','')
s_3 = s_3.replace('No','')
re_comp  = re.compile('[A-Z]{3}[0-9]{3}') # 과목코드 제거
skk = re_comp.sub('',skk)
s_3 = re_comp.sub('', s_3)
skk = re.sub('[0-9]*[0-9]','',skk) # 이수 학기 제거
s_3 = re.sub('[0-9]*[0-9]','',s_3)
skk = re.sub("[0-9]*' '*[0-9]",'',skk) # 학점 제거
s_3 = re.sub("[0-9]*' '*[0-9]",'',s_3)
driver.close()

# 과정소개  - 준비중


## 2-0. 특수기호 제거

In [23]:
cau = re.sub('[(-:*→•)]',' ',cau)
skk = re.sub('[(-:*→•\n)]',' ',skk)
yon = re.sub('[(-:*→•▹ㆍ\n)]',' ',yon)

In [24]:
all_texts = []
all_texts.append(kom.nouns(cau))
all_texts.append(kom.nouns(skk))
all_texts.append(kom.nouns(yon))

## 2-1. 토큰화 (명사 추출)

In [25]:
cau_docs = []
cau_docs.append(kom.nouns(c_1))
cau_docs.append(kom.nouns(c_2))
cau_docs.append(kom.nouns(c_3))

In [26]:
yon_docs = []
y_1 = re.sub('[(-:*→•▹ㆍ\n)]',' ',y_1)
y_2 = re.sub('[(-:*→•▹ㆍ\n\t\r)]',' ',y_2)
y_3 = re.sub('[(-:*→•▹ㆍ\n)]',' ',y_3)

yon_docs.append(kom.nouns(y_1))
yon_docs.append(kom.nouns(y_2))
yon_docs.append(kom.nouns(y_3))

In [27]:
s_1 =s_1.replace('\n',' ')
s_2 =s_2.replace('\n',' ')
s_3 =s_3.replace('\n',' ')

skk_docs = []
skk_docs.append(kom.nouns(s_1))
skk_docs.append(kom.nouns(s_2))
skk_docs.append(kom.nouns(s_3))

## 2-2. 불용어 제거

### 2-2-2. 불용어 제거
- 한 글자 단어 위주로 제거 ('웹' 은 예외로 함)
- 문헌정보학, 문헌정보, 문헌정보학과 삭제
- 큰 의미 없이 반복되는 단어('전선','전공 등) 삭제
- 타 토큰들과 비교했을 때, 이상치 수준으로 빈도수가 큰 경우(정보)

In [104]:
stopwords = ['문헌정보학','문헌정보','문헌정보학과','전선','전공','관련','정보','대학']

In [105]:
cau_t_list = []
skk_t_list = []
yon_t_list = []

# 각 문서별로 불용어 제거한 후 리스트에 추가하는 함수
def appending_non_stopwords(vacant_list, word_list):
    for t in word_list:
        if len(t) > 1 and t not in stopwords:
            vacant_list.append(t)
    return vacant_list



In [106]:
# 중앙대학교
c_1_n = []
c_2_n = []
c_3_n = []
c_v_list = [c_1_n, c_2_n, c_3_n]
for i in range(len(cau_docs)):
    c_v_list[i] = appending_non_stopwords(c_v_list[i],cau_docs[i])


In [107]:
# 연세대학교
y_1_n = []
y_2_n = []
y_3_n = []
y_v_list = [y_1_n, y_2_n, y_3_n]
for i in range(len(yon_docs)):
    y_v_list[i] = appending_non_stopwords(y_v_list[i],yon_docs[i])

In [108]:
# 성균관대학교  
s_1_n = []
s_2_n = []
s_3_n = []
s_v_list = [s_1_n, s_2_n, s_3_n]
for i in range(len(skk_docs)):
    s_v_list[i] = appending_non_stopwords(s_v_list[i],skk_docs[i])

# 3. Topic Modeling

- 학과별 홈페이지에서 개별 항목별 페이지(연혁, 소개, 이수 과목 ... etc)를 개별 문서로 보고 진행.

### 3-1. 텍스트 정수 인코딩

In [109]:
# 중앙대학교
dictionary_cau = corpora.Dictionary(c_v_list)
corpus_cau = [dictionary_cau.doc2bow(text) for text in c_v_list]

print(len(dictionary_cau))
print(corpus_cau[1])

432
[(5, 1), (7, 1), (8, 1), (9, 1), (10, 1), (11, 1), (16, 1), (17, 1), (18, 2), (21, 1), (24, 1), (25, 5), (27, 2), (28, 1), (30, 1), (34, 1), (41, 1), (43, 1), (45, 1), (47, 1), (48, 4), (54, 3), (59, 1), (64, 1), (65, 1), (66, 2), (70, 1), (71, 1), (73, 1), (76, 4), (77, 1), (78, 1), (80, 6), (81, 1), (84, 2), (85, 1), (86, 13), (87, 1), (88, 1), (89, 1), (92, 2), (93, 1), (95, 1), (96, 1), (99, 1), (101, 1), (103, 1), (105, 1), (106, 1), (109, 1), (110, 4), (115, 7), (117, 1), (120, 3), (122, 2), (125, 1), (126, 1), (127, 2), (128, 1), (130, 2), (133, 1), (134, 2), (135, 1), (136, 2), (137, 3), (138, 1), (139, 1), (140, 2), (144, 1), (146, 2), (151, 3), (152, 2), (154, 1), (158, 6), (159, 1), (160, 1), (165, 6), (169, 1), (170, 1), (172, 1), (173, 1), (176, 2), (185, 1), (186, 1), (187, 2), (189, 1), (190, 6), (192, 1), (193, 1), (197, 2), (198, 1), (199, 2), (200, 4), (202, 1), (203, 1), (205, 1), (206, 2), (207, 1), (208, 2), (209, 1), (210, 4), (211, 2), (212, 1), (213, 1), (21

In [110]:
# 연세대학교
dictionary_yon = corpora.Dictionary(y_v_list)
corpus_yon = [dictionary_yon.doc2bow(text) for text in y_v_list]

print(len(dictionary_yon))
print(corpus_yon[0])

386
[(0, 3), (1, 4), (2, 1), (3, 1), (4, 1), (5, 5), (6, 1), (7, 4), (8, 1), (9, 6), (10, 4), (11, 2), (12, 1), (13, 1), (14, 3), (15, 2), (16, 1), (17, 2), (18, 1), (19, 3), (20, 3), (21, 5), (22, 3), (23, 1), (24, 2), (25, 1), (26, 1), (27, 5), (28, 2), (29, 5), (30, 1), (31, 3), (32, 2), (33, 1), (34, 9), (35, 2), (36, 1), (37, 1), (38, 4), (39, 1), (40, 1), (41, 3), (42, 2), (43, 9), (44, 1), (45, 1), (46, 1), (47, 1), (48, 2), (49, 1), (50, 4), (51, 2), (52, 3), (53, 1), (54, 6), (55, 2), (56, 5), (57, 1), (58, 1), (59, 1), (60, 3), (61, 2), (62, 1), (63, 1), (64, 1), (65, 1), (66, 1), (67, 6), (68, 4), (69, 4), (70, 1), (71, 2), (72, 8), (73, 1), (74, 7), (75, 1), (76, 3), (77, 1), (78, 1), (79, 4), (80, 4), (81, 1), (82, 1), (83, 1), (84, 3), (85, 6), (86, 1), (87, 2), (88, 5), (89, 1), (90, 1), (91, 2), (92, 2), (93, 1), (94, 2), (95, 1), (96, 1), (97, 1), (98, 1), (99, 1), (100, 1), (101, 3), (102, 4), (103, 1), (104, 6), (105, 9), (106, 2), (107, 2), (108, 3), (109, 5), (110,

In [111]:
# 성균관대학교
dictionary_skk = corpora.Dictionary(s_v_list)
corpus_skk = [dictionary_skk.doc2bow(text) for text in s_v_list]

print(len(dictionary_skk))
print(corpus_skk[1])

264
[(1, 1), (15, 2), (21, 3), (24, 1), (25, 1), (26, 1), (35, 1), (37, 5), (38, 2), (40, 1), (44, 2), (46, 1), (49, 1), (51, 1), (52, 1), (53, 1), (56, 1), (57, 2), (60, 1), (61, 2), (66, 1), (68, 3), (76, 1), (77, 2), (78, 1), (79, 3), (81, 1), (85, 1), (90, 2), (93, 3), (96, 4), (103, 1), (104, 1), (105, 1), (106, 1), (107, 2), (108, 1), (109, 1), (110, 1), (111, 1), (112, 1), (113, 1), (114, 2), (115, 1), (116, 1), (117, 4), (118, 1), (119, 1), (120, 1), (121, 1), (122, 1), (123, 1), (124, 1), (125, 1), (126, 1), (127, 1), (128, 1), (129, 1), (130, 1), (131, 1), (132, 1), (133, 1), (134, 1), (135, 1), (136, 2), (137, 1), (138, 1), (139, 1), (140, 1), (141, 1), (142, 1), (143, 1), (144, 1), (145, 6), (146, 1), (147, 3), (148, 1), (149, 1), (150, 1), (151, 1), (152, 1), (153, 1), (154, 1), (155, 4), (156, 1), (157, 1), (158, 2), (159, 1), (160, 1), (161, 1), (162, 1), (163, 1), (164, 1), (165, 1), (166, 1), (167, 4), (168, 1), (169, 1), (170, 3), (171, 2), (172, 1), (173, 2), (174, 1

### 3-2. LDA 모델 훈련
- 세 대학 모두, 토픽 개수를 10개로 설정 했을 때, 토픽에 따른 단어 그룹이 크게 2개 정도 나오고, 나머지 8개 토픽은 거의 의미가 없는 수준으로 적은 단어로 형성된 토픽 그룹이 생성됨.
- 따라서, 토픽의 개수를 2개로 설정하여 다시 훈련시켰음.
- 학습 토픽 수가 적고, 대체적으로 모든 문서에서 사용되는 단어 집합이 비슷하여 명확히 토픽이 구분되지 않는 현상 발생.

__(1) 중앙대학교__

In [112]:
ldamodel_cau = gensim.models.ldamodel.LdaModel(corpus_cau, num_topics = 2,
 id2word = dictionary_cau, passes = 15 )
topics_cau = ldamodel_cau.print_topics(num_words = 30)
for topic in topics_cau:
    print(topic)

(0, '0.022*"사회" + 0.017*"양성" + 0.012*"연구" + 0.012*"운영" + 0.010*"중앙대학교" + 0.010*"처리" + 0.010*"학문" + 0.010*"분야" + 0.010*"데이터" + 0.009*"교육" + 0.009*"지식" + 0.009*"중심" + 0.009*"전문가" + 0.007*"도서관" + 0.007*"실습" + 0.007*"현상" + 0.007*"변화" + 0.007*"융합" + 0.007*"경쟁력" + 0.007*"바탕" + 0.006*"조직" + 0.006*"목표" + 0.006*"자료" + 0.006*"전국" + 0.006*"영역" + 0.006*"능력" + 0.006*"컴퓨터" + 0.006*"현장" + 0.006*"이론" + 0.006*"기여"')
(1, '0.020*"학점" + 0.015*"졸업" + 0.015*"도서관" + 0.012*"학과" + 0.012*"개최" + 0.011*"필수" + 0.011*"사회" + 0.011*"이수" + 0.010*"학기" + 0.010*"체육" + 0.010*"학술" + 0.010*"대회" + 0.008*"과정" + 0.008*"우승" + 0.007*"현상" + 0.007*"문헌" + 0.007*"기준" + 0.007*"과학" + 0.007*"관리" + 0.007*"동문" + 0.007*"변경" + 0.006*"영역" + 0.006*"포함" + 0.006*"실습" + 0.006*"학문" + 0.006*"일반" + 0.006*"사서" + 0.006*"사항" + 0.006*"대학원" + 0.006*"개론"')


- topic 0에는 '학점'이나 '졸업', '필수', '학년' 등 교과 과정에 대한 단어가 다수 나타난다. 따라서 topic 0은 '교과과정'이라고 볼 수 있다.
- topic 1에는 topic 0 보다는 다양한 종류의 단어가 다수 포함되어 있어, 명확히 어떤 주제라고 상정하긴 어려우나, 대체적으로 '사회' 나 '학문', '데이터', '대학원', '동문', '전문가' 등 진로에 관한 단어가 다수 포함되어 있으므로, topic 1을 '진로' 라고 볼 수 있다.

__(2) 연세대학교__

In [113]:
ldamodel_yon = gensim.models.ldamodel.LdaModel(corpus_yon, num_topics = 2,
 id2word = dictionary_yon, passes = 15 )
topics_yon = ldamodel_yon.print_topics(num_words = 30)
for topic in topics_yon:
    print(topic)

(0, '0.015*"지식" + 0.015*"현상" + 0.014*"도서관" + 0.014*"이론" + 0.012*"능력" + 0.012*"영역" + 0.011*"조직" + 0.011*"서비스" + 0.011*"색인" + 0.010*"응용" + 0.010*"연구" + 0.010*"융합" + 0.008*"기반" + 0.008*"분류" + 0.008*"정보기술" + 0.008*"교육" + 0.008*"검색" + 0.008*"문제" + 0.008*"자원" + 0.008*"시대" + 0.007*"시스템" + 0.007*"요소" + 0.007*"제공" + 0.007*"개발" + 0.007*"커뮤니케이션" + 0.007*"컴퓨터" + 0.007*"문헌" + 0.007*"기술" + 0.007*"이해" + 0.007*"해결"')
(1, '0.019*"분석" + 0.018*"도서관" + 0.018*"평가" + 0.014*"이론" + 0.013*"서비스" + 0.013*"관리" + 0.013*"능력" + 0.013*"사회" + 0.011*"실제" + 0.011*"센터" + 0.011*"시스템" + 0.011*"기법" + 0.011*"처리" + 0.010*"기술" + 0.010*"방법" + 0.010*"업무" + 0.009*"조직" + 0.009*"적용" + 0.009*"기본" + 0.009*"개념" + 0.008*"데이터" + 0.008*"커뮤니케이션" + 0.008*"자료" + 0.008*"학술" + 0.008*"기능" + 0.008*"선택" + 0.008*"시각" + 0.007*"교육" + 0.007*"이해" + 0.007*"개발"')


- topic 0에는 '색인' ,'조직' , '분류', '검색' 등 자료조직과 관련된 단어가 다수 포함됨. 따라서 topic 0을 '자료조직'으로 봄.
- topic 1에는 '분석', '처리', '데이터' 등 데이터 사이언스와 관련된 단어가 다수 포함됨. 따라서 topic 1을 '데이터사이언스'로 봄

__(3) 성균관대학교__

In [114]:
ldamodel_skk = gensim.models.ldamodel.LdaModel(corpus_skk, num_topics = 2,
 id2word = dictionary_skk, passes = 15 )
topics_skk = ldamodel_skk.print_topics(num_words = 30)
for topic in topics_skk:
    print(topic)

(0, '0.150*"학사" + 0.096*"데이터" + 0.071*"사이언스" + 0.049*"융합" + 0.036*"실습" + 0.027*"현장" + 0.021*"학기" + 0.021*"도전" + 0.019*"연구" + 0.017*"학점" + 0.016*"인문학" + 0.016*"인공지능" + 0.012*"석사" + 0.012*"개론" + 0.011*"박사" + 0.011*"대학원" + 0.011*"문화" + 0.011*"문과" + 0.011*"창업" + 0.007*"국제" + 0.007*"분석" + 0.007*"비교" + 0.007*"활용" + 0.007*"과학" + 0.007*"하계" + 0.007*"컴퓨터" + 0.006*"교육" + 0.006*"응용" + 0.004*"글로벌" + 0.004*"사회과학"')
(1, '0.047*"교수" + 0.023*"학과" + 0.021*"부임" + 0.021*"3월" + 0.012*"도서관" + 0.012*"분야" + 0.012*"데이터" + 0.010*"글로벌" + 0.010*"정원" + 0.010*"정년" + 0.010*"과정" + 0.008*"일반" + 0.008*"퇴임" + 0.008*"영역" + 0.008*"명예" + 0.008*"본교" + 0.008*"창설" + 0.008*"시대" + 0.008*"전문가" + 0.008*"8월" + 0.008*"특성" + 0.008*"프로그램" + 0.008*"주년" + 0.008*"천혜" + 0.008*"최초" + 0.008*"추대" + 0.008*"연수" + 0.008*"교육" + 0.008*"박사" + 0.008*"연구"')


- topic 0에는 '현장' , '대학원' , '박사' , '석사', '창업' 등 졸업 이후 진로와 관련된 단어가 다수 포함되어 있음. 따라서 topic 0을 '진로'라고 상정함.
- topic 2에는 '교수' , '부임', '정년' , '명예' 등 교수에 관한 단어들이 다수 포함되어 있음. 따라서 topic 2를 '교수'라고 상정함.

### 3-3. LDA 시각화 및 word cloud와의 비교분석

- 문서 별 개별 토픽의 비중이 매우 높게 나옴. 이는 학습시킨 문서의 수와 문서 내의 단어 수가 적어서 발생한 현상으로 보임.

__(1) 중앙대학교__

In [115]:
import pyLDAvis.gensim_models

pyLDAvis.enable_notebook()
vis_cau = pyLDAvis.gensim_models.prepare(ldamodel_cau, corpus_cau, dictionary_cau)
pyLDAvis.display(vis_cau)

In [116]:
for i, topic_list in enumerate(ldamodel_cau[corpus_cau]):
    if i==4:
        break
    print(i,'번째 문서의 topic 비율은',topic_list)

0 번째 문서의 topic 비율은 [(1, 0.99799573)]
1 번째 문서의 topic 비율은 [(0, 0.9985039)]
2 번째 문서의 topic 비율은 [(1, 0.9975462)]


[cauimg](./img/cauwc.png)

__(2) 연세대학교__

In [117]:
pyLDAvis.enable_notebook()
vis_yon = pyLDAvis.gensim_models.prepare(ldamodel_yon, corpus_yon, dictionary_yon)
pyLDAvis.display(vis_yon)

In [118]:
for i, topic_list in enumerate(ldamodel_yon[corpus_yon]):
    if i==4:
        break
    print(i,'번째 문서의 topic 비율은',topic_list)

0 번째 문서의 topic 비율은 [(0, 0.998713)]
1 번째 문서의 topic 비율은 [(0, 0.9881487), (1, 0.011851312)]
2 번째 문서의 topic 비율은 [(1, 0.9989667)]


__(3) 성균관대학교__

In [119]:
pyLDAvis.enable_notebook()
vis_skk = pyLDAvis.gensim_models.prepare(ldamodel_skk, corpus_skk, dictionary_skk)
pyLDAvis.display(vis_skk)

In [120]:
for i, topic_list in enumerate(ldamodel_skk[corpus_skk]):
    if i==4:
        break
    print(i,'번째 문서의 topic 비율은',topic_list)

0 번째 문서의 topic 비율은 [(1, 0.99757236)]
1 번째 문서의 topic 비율은 [(1, 0.9965132)]
2 번째 문서의 topic 비율은 [(0, 0.99888927)]
